In [28]:
!scala -version



Scala code runner version 2.12.10 -- Copyright 2002-2019, LAMP/EPFL and Lightbend, Inc.


In [29]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('BigQuery Storage & Spark DataFrames') \
    .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
    .getOrCreate()

In [30]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
!spark -version

/usr/bin/sh: 1: spark: not found


In [31]:
table = "bigquery-public-data.wikipedia.pageviews_2020"
df_wiki_pageviews = spark.read \
    .format("bigquery") \
    .option("table", table) \
    .option("filter", "datehour >= '2020-03-01' AND datehour < '2020-03-02'") \
    .load()

df_wiki_pageviews.printSchema()

root
 |-- datehour: timestamp (nullable = true)
 |-- wiki: string (nullable = true)
 |-- title: string (nullable = true)
 |-- views: long (nullable = true)



In [32]:
'''\
I got a fright from how many rows of the column-title was empty. So I added two more conditions in the .where clause which filtered out the rows with empty titles'''

df_wiki_en = df_wiki_pageviews \
  .select("title", "wiki", "views") \
  .where("views > 1000 AND wiki in ('en', 'en.m') AND title IS NOT NULL AND title != '-'") \
  .cache()

df_wiki_en.show()


+--------------------+----+-----+
|               title|wiki|views|
+--------------------+----+-----+
|12_Angry_Men_(195...|  en| 1124|
|Apple_Network_Server|  en| 1098|
|2020_South_Caroli...|  en| 1864|
|            Brooklyn|  en| 1147|
|Altered_Carbon_(T...|  en| 1421|
|2020_coronavirus_...|  en| 2897|
|2020_Democratic_P...|  en| 3932|
|      Bernie_Sanders|  en| 1244|
|2019–20_Wuhan_cor...|  en| 1160|
|2019–20_coronavir...|  en| 8864|
|               Bible|  en| 1304|
|       Boris_Johnson|  en| 1007|
|2019–20_coronavir...|  en| 2390|
|      Bernie_Sanders|  en| 1580|
|               Bible|  en| 5411|
|            Brooklyn|  en| 1652|
|2019–20_coronavir...|  en| 1291|
|Altered_Carbon_(T...|  en| 1569|
|Bombshell_(2019_f...|  en| 1506|
|2020_Democratic_P...|  en| 6971|
+--------------------+----+-----+
only showing top 20 rows



In [33]:
import pyspark.sql.functions as F

df_wiki_en_totals = df_wiki_en \
.groupBy("title") \
.agg(F.sum('views').alias('total_views'))

df_wiki_en_totals.orderBy('total_views', ascending=False)

title,total_views
Main_Page,10939337
United_States_Senate,5619797
Special:Search,1538334
2019–20_coronavir...,407042
2020_Democratic_P...,260093
Coronavirus,254861
The_Invisible_Man...,233718
Super_Tuesday,201077
Colin_McRae,200219
David_Byrne,189989


In [43]:
# Update to your GCS bucket

gcs_bucket = 'eben-lab4'

# Update to your BigQuery dataset name you created
bq_dataset = 'my-spark-project-442210.new_dataset'

# Enter BigQuery table name you want to create or overwrite. 
# If the table does not exist it will be created when you run the write function
bq_table = 'wiki_total_pageviews'

df_wiki_en_totals.write \
  .format("bigquery") \
  .option("table", "{}.{}".format(bq_dataset, bq_table)) \
  .option("temporaryGcsBucket", gcs_bucket) \
  .mode('overwrite') \
  .save()